In [19]:
import tweepy
import time
import argparse
import string
import json
import os
import sys
import re
import operator 
import gensim
import pprint
import unicodedata
import langid
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from collections import defaultdict
from gensim import corpora
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn import metrics
from sklearn.cluster import KMeans
from gensim import corpora, models
import pyLDAvis.gensim

In [9]:
fname = 'Twitter/data/Data_backup/stream_Empire.json'
texts = []
count  = 0
with open(fname, 'r') as f:
    for line in f:
        try:
            tweet = json.loads(line)
            if '#empire' in tweet['text'].lower():
                texts.append(tweet)
        except:
            continue

print len(texts)

fname = 'Twitter/data/Data_backup/stream__Empire.json'
with open(fname, 'r') as f:
    for line in f:
        try:
            tweet = json.loads(line)
            texts.append(tweet)
        except:
            continue
            
print len(texts)

fname = 'Twitter/data/stream__Empire.json'
with open(fname, 'r') as f:
    for line in f:
        try:
            tweet = json.loads(line)
            texts.append(tweet)
        except:
            continue

print len(texts)

23556
121095
182822


In [10]:
impTexts = []
for tweet in texts:
    if "created_at" in tweet and 'Dec' in tweet['created_at'] and '15' in tweet['created_at'][8:10]:
        impTexts.append(tweet)

print len(impTexts)

77803


In [12]:
emoticons_str = r"""
	(?:
		[:=;] # Eyes
		[oO\-]? # Nose (optional)
		[D\)\]\(\]/\\OpP] # Mouth
	)"""
 
regex_str = [
	emoticons_str,
	r'<[^>]+>', # HTML tags
	r'(?:@[\w_]+)', # @-mentions
	r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
	r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
	r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
	r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
	r'(?:[\w_]+)', # other words
	r'(?:\S)' # anything else
]
	
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
	return tokens_re.findall(s)

def filter_lang(lang, document):

	doclang = langid.classify(document)
	return doclang[0]
	##return [documents[k] for k in range(len(documents)) if doclang[k][0] == lang

def preprocess(s, lowercase=False):

	tokens = tokenize(s)
	if lowercase:
		tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
	return tokens

# Remove stop words
stoplist_tw=['amp','get','got','hey','hmm','hoo','hop','iep','let','ooo','par',
			'pdt','pln','pst','wha','yep','yer','yet','aest','didn','nzdt','via',
			'one','com','new','like','great','make','top','awesome','best',
			'good','wow','yes','say','yay','would','thanks','thank','going',
			'new','use','should','could','best','really','see','want','nice',
			'while','know', 'rt', 'via', 'it', 'u', 'c', 'co', 'http', 'w', 'r', '1', '2', '3',
			'gt', 'v', 'It', 'e', 'k', 'gg']

punctuation = list(string.punctuation)
stop = set(stopwords.words('english') + stoplist_tw) 
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()


def clean(text):
	text=re.sub('[^a-z0-9 ]',' ',text.lower())
	text=re.sub('  +',' ', text).strip()
	stop_free = [i for i in text.split(' ') if i not in stop]
	normalized = [lemma.lemmatize(word) for word in stop_free]
	return ' '.join(normalized)


In [13]:
ldaTexts = []
for tweet in impTexts:
    if filter_lang('en', tweet['text']) == "en":
        ldaTexts.append(clean(tweet['text']).split())

In [15]:
token_frequency = defaultdict(int)
# count all token
for doc in ldaTexts:
    for token in doc:
        token_frequency[token] += 1

In [16]:
# keep words that occur more than once
cleanTexts = [ [token for token in doc if token_frequency[token] > 1] for doc in ldaTexts ]

In [17]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index.
dictionary = corpora.Dictionary(cleanTexts)
dictionary.save('Twitter/output/dictionary.dict')

bow_corpus = [dictionary.doc2bow(doc) for doc in cleanTexts]
corpora.MmCorpus.serialize('Twitter/output/corpus.mm', bow_corpus)

In [18]:
## apply LDA
lda = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics=3, id2word = dictionary, passes=20)
lda_corpus = lda[bow_corpus]

lda.save('Twitter/output/document.lda')
print("lda saved in %s " % 'Twitter/output/document.lda')

lda saved in Twitter/output/document.lda 


In [21]:
corpus = corpora.MmCorpus('Twitter/output/corpus.mm')
dictionary = corpora.Dictionary.load('Twitter/output/dictionary.dict')

In [31]:
lda = models.LdaModel.load('Twitter/output/document.lda')
topic_data =  pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(topic_data)
pyLDAvis.save_html(topic_data, 'Twitter/output/LdaModel.html')

In [23]:
lda.show_topics()

[(0,
  u'0.111*"empire" + 0.044*"andre" + 0.029*"http" + 0.025*"rhonda" + 0.024*"bet" + 0.019*"hakeem" + 0.016*"take" + 0.015*"oh" + 0.015*"lucious" + 0.014*"heart"'),
 (1,
  u'0.144*"http" + 0.104*"empire" + 0.059*"empirefox" + 0.047*"star" + 0.022*"finale" + 0.019*"felizjueves" + 0.016*"fall" + 0.010*"tonight" + 0.009*"retweet" + 0.008*"leedanielsent"'),
 (2,
  u'0.124*"empire" + 0.042*"jamal" + 0.026*"cookie" + 0.019*"jussiesmollett" + 0.016*"lucious" + 0.015*"empirefox" + 0.011*"luscious" + 0.011*"rehab" + 0.010*"go" + 0.009*"empirewednesday"')]

In [24]:
## apply LDA
lda = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics=10, id2word = dictionary, passes=20)
lda_corpus = lda[bow_corpus]

lda.save('Twitter/output/document_t10.lda')
print("lda saved in %s " % 'Twitter/output/document_t10.lda')

lda saved in Twitter/output/document_t10.lda 


In [25]:
lda = models.LdaModel.load('Twitter/output/document_t10.lda')
topic_data =  pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(topic_data)

In [27]:
lda.show_topics()

[(0,
  u'0.112*"empire" + 0.079*"bet" + 0.049*"time" + 0.026*"crazy" + 0.020*"tariq" + 0.017*"angelo" + 0.015*"epic" + 0.015*"as" + 0.015*"enough" + 0.014*"10"'),
 (1,
  u'0.122*"andre" + 0.115*"empire" + 0.070*"rhonda" + 0.050*"take" + 0.042*"oh" + 0.027*"cookie" + 0.021*"think" + 0.020*"tarhondajay" + 0.019*"never" + 0.015*"want"'),
 (2,
  u'0.095*"http" + 0.093*"empire" + 0.077*"star" + 0.031*"finale" + 0.030*"heart" + 0.029*"start" + 0.027*"neweditionbet" + 0.025*"january" + 0.025*"24th" + 0.025*"l0d0ofuflq"'),
 (3,
  u'0.104*"empire" + 0.086*"empirefox" + 0.082*"http" + 0.046*"lucious" + 0.039*"cookie" + 0.031*"leedanielsent" + 0.029*"anika" + 0.025*"kill" + 0.023*"8" + 0.020*"7c"'),
 (4,
  u'0.167*"empire" + 0.149*"jamal" + 0.038*"rehab" + 0.034*"empirefox" + 0.030*"takeem" + 0.024*"jussiesmollett" + 0.021*"go" + 0.018*"shook" + 0.018*"empirewednesday" + 0.017*"tiana"'),
 (5,
  u'0.223*"http" + 0.124*"empire" + 0.120*"empirefox" + 0.028*"retweet" + 0.023*"finale" + 0.021*"fall" +

In [28]:
## apply LDA
lda = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics=5, id2word = dictionary, passes=20)
lda_corpus = lda[bow_corpus]

lda.save('Twitter/output/document_t5.lda')
print("lda saved in %s " % 'Twitter/output/document_t5.lda')

lda saved in Twitter/output/document_t5.lda 


In [29]:
lda = models.LdaModel.load('Twitter/output/document_t5.lda')
topic_data =  pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(topic_data)

In [30]:
lda.show_topics()

[(0,
  u'0.110*"empire" + 0.083*"http" + 0.039*"rhonda" + 0.038*"bet" + 0.025*"andre" + 0.024*"empirefox" + 0.022*"heart" + 0.022*"oh" + 0.020*"neweditionbet" + 0.019*"building"'),
 (1,
  u'0.104*"empire" + 0.034*"hakeem" + 0.033*"take" + 0.031*"andre" + 0.027*"lucious" + 0.017*"crazy" + 0.013*"think" + 0.012*"cold" + 0.012*"love" + 0.010*"back"'),
 (2,
  u'0.121*"empire" + 0.021*"andre" + 0.020*"luscious" + 0.016*"cookie" + 0.016*"march" + 0.016*"damn" + 0.015*"lucious" + 0.014*"wait" + 0.013*"pill" + 0.012*"come"'),
 (3,
  u'0.139*"empire" + 0.082*"http" + 0.080*"empirefox" + 0.069*"jamal" + 0.024*"cookie" + 0.022*"retweet" + 0.018*"lucious" + 0.018*"jussiesmollett" + 0.018*"rehab" + 0.016*"kill"'),
 (4,
  u'0.126*"http" + 0.099*"empire" + 0.062*"star" + 0.035*"empirefox" + 0.028*"finale" + 0.025*"felizjueves" + 0.020*"fall" + 0.013*"tonight" + 0.010*"leedanielsent" + 0.009*"time"')]